# Tutorial 04: Long-Term Ordering (aKMC)

**Goal**: Overcome the time-scale limitation of standard MD to observe chemical ordering in FePt.

This tutorial demonstrates:
1.  **Rare Event Sampling**: Using Adaptive Kinetic Monte Carlo (aKMC) to find transition states.
2.  **Long-Timescale Evolution**: Observing the transformation from a disordered alloy to the L1_0 ordered phase.
3.  **EON Integration**: How the orchestrator manages external aKMC codes.



In [ ]:
import os
import shutil
from pathlib import Path
from unittest.mock import patch

from ase.io import read
from ase.visualize import view

from mlip_autopipec.core.orchestrator import Orchestrator
from mlip_autopipec.domain_models.config import (
    GlobalConfig,
    EONDynamicsConfig,
    AdaptiveGeneratorConfig,
    QEOracleConfig,
    PacemakerTrainerConfig,
    StandardValidatorConfig,
    MockGeneratorConfig,
    MockOracleConfig,
    MockTrainerConfig,
    MockDynamicsConfig,
    MockValidatorConfig,
    PhysicsBaselineConfig
)
from mlip_autopipec.domain_models.enums import (
    GeneratorType,
    OracleType,
    TrainerType,
    DynamicsType,
    ValidatorType,
)
from mlip_autopipec.domain_models.structure import Structure



## 1. Setup Environment

We look for the `eon` executable.



In [ ]:
def has_command(cmd):
    return shutil.which(cmd) is not None

HAS_EON = has_command("eon")
# Check previous tutorial output
NB03_OUTPUT = Path("workdirs/03_deposition/final_structure.xyz").resolve()

IS_CI_MODE = os.environ.get("IS_CI_MODE", "False").lower() == "true"
if not HAS_EON:
    print("EON not found. Forcing CI Mode.")
    IS_CI_MODE = True

if not NB03_OUTPUT.exists() and not IS_CI_MODE:
    print("Previous tutorial output not found. Please run Tutorial 03 first.")
    # In a real notebook we might stop here. For testing script, we fallback to CI logic if possible.
    if not IS_CI_MODE:
         # Create dummy file to allow script to proceed if user insists on Real mode but missed a step
         # (Though usually we should fail)
         pass

WORKDIR = Path("workdirs/04_ordering")
if WORKDIR.exists():
    shutil.rmtree(WORKDIR)
WORKDIR.mkdir(parents=True, exist_ok=True)



## 2. Load Disordered Structure

We load the final state from the PVD simulation.



In [ ]:
if NB03_OUTPUT.exists():
    # Load the structure
    atoms = read(NB03_OUTPUT)
    # Ensure it has P1 symmetry for EON to find processes
    # (Symmetry breaking is usually key)
    atoms.info["type"] = "cluster"
    start_structure = Structure.from_ase(atoms)
    print(f"Loaded structure with {len(atoms)} atoms.")
else:
    # Create a dummy structure for CI/Mock
    from ase.build import bulk
    atoms = bulk("Pt", "fcc", a=3.9).repeat((2, 2, 2))
    start_structure = Structure.from_ase(atoms)
    print("Created dummy structure (CI Mode).")



## 3. Configure Orchestrator for aKMC

We replace the standard MD engine with `EONDynamics`.
The Orchestrator will:
1.  Take the seed structure (our disordered cluster).
2.  Run EON to find saddle points and evolve the system.
3.  If EON encounters a state with high uncertainty (OTF check), it halts.
4.  The Orchestrator then triggers the Oracle (DFT) to label that state.



In [ ]:
# Define custom generator to inject the start structure
def custom_generator_func(self, n_structures: int, cycle: int = 0, metrics = None):
    print(f"Injecting start structure for aKMC (Cycle {cycle})")
    yield start_structure

def create_config(workdir: Path, is_ci: bool) -> GlobalConfig:
    if is_ci:
        return GlobalConfig(
            workdir=workdir,
            max_cycles=1,
            components={
                "generator": MockGeneratorConfig(
                    name=GeneratorType.MOCK,
                    n_structures=1,
                    cell_size=10.0,
                    n_atoms=len(start_structure.positions),
                    atomic_numbers=start_structure.atomic_numbers
                ),
                "oracle": MockOracleConfig(name=OracleType.MOCK),
                "trainer": MockTrainerConfig(name=TrainerType.MOCK),
                "dynamics": MockDynamicsConfig(
                    name=DynamicsType.MOCK,
                    selection_rate=0.5,
                    simulated_uncertainty=5.0
                ),
                "validator": MockValidatorConfig(name=ValidatorType.MOCK)
            }
        )
    else:
        return GlobalConfig(
            workdir=workdir,
            max_cycles=5, # Allow multiple active learning cycles
            components={
                "generator": AdaptiveGeneratorConfig(
                    name=GeneratorType.ADAPTIVE,
                    n_structures=1,
                    element="FePt", # Placeholder
                    crystal_structure="L1_0", # Placeholder
                ),
                "oracle": QEOracleConfig(
                    name=OracleType.QE,
                    ecutwfc=50.0,
                    ecutrho=400.0,
                    pseudopotentials={
                        "Fe": "Fe.pbe-spn-kjpaw_psl.1.0.0.UPF",
                        "Pt": "Pt.pbe-n-kjpaw_psl.1.0.0.UPF",
                        "Mg": "Mg.pbe-n-kjpaw_psl.1.0.0.UPF", # In case substrate atoms are present
                        "O": "O.pbe-n-kjpaw_psl.1.0.0.UPF"
                    }
                ),
                "trainer": PacemakerTrainerConfig(
                    name=TrainerType.PACEMAKER,
                    cutoff=5.0,
                    max_num_epochs=50,
                    # initial_potential="workdirs/02_interface/cycle_01/potential.yace"
                ),
                "dynamics": EONDynamicsConfig(
                    name=DynamicsType.EON,
                    temperature=600.0,
                    time_step=1.0, # EON time step is usually arbitrary for KMC
                    n_events=1000,
                    uncertainty_threshold=0.1 # Very strict for aKMC
                ),
                "validator": StandardValidatorConfig(
                    name=ValidatorType.STANDARD
                )
            }
        )

config = create_config(WORKDIR, IS_CI_MODE)



### Run Orchestrator

We patch the generator to start the loop with our structure.



In [ ]:
target_class = "mlip_autopipec.components.generator.adaptive.AdaptiveGenerator"
if IS_CI_MODE:
    target_class = "mlip_autopipec.components.generator.mock.MockGenerator"

with patch(f"{target_class}.generate", custom_generator_func):
    orchestrator = Orchestrator(config)
    orchestrator.run()

print("aKMC Loop Complete.")



## 4. Analysis

We check if the system found a lower energy state.
In a real tutorial, we would parse EON's `energy.dat` or similar output.



In [ ]:
eon_log = WORKDIR / "cycle_01" / "eon_run_00000" / "client.log"
if IS_CI_MODE:
    # Mock validation
    print("CI Mode: Skipping EON log analysis.")
else:
    if eon_log.exists():
        print("Found EON log. Simulation ran.")
        # Analyze order parameter (count Fe-Pt bonds)
        # For now, just print success.
    else:
        print("EON log not found. Did it run?")
